# Demonstrate the "target-decoy" approach, as applied to metagenomic variant calling

In [1]:
%run "Header.ipynb"

In [7]:
import skbio
import pileup
from parse_sco import parse_sco

In [2]:
seq2pos2pileup = pileup.load()

In [5]:
m = 5
percentages = [50, 25, 10, 5, 2, 1, 0.5, 0.25, 0.1, 0.05]
p2pct = get_p2pct(percentages)
p2mincov = get_p2mincov(percentages, m)

In [6]:
# Maps seq name -> list of 1-indexed positions where we "call" a mutation
seq2naively_called_mutations = {}
seq2lofreq_called_mutations = {}

In [ ]:
# Naively call mutations
for seq in SEQS:
    positions = range(1, seq2len[seq] + 1)
    for pos in positions:
        pass
    
# Find mutations in CAMP
# Compute frac_decoy
# Estimate FDR of mutations in other genomes